In [1]:
import lib.NotationLinear as nlin
import lib.score_comparison_lib as scl

import music21 as m21
from pathlib import Path

In [2]:
score1_path = Path("test_scores/musicxml/longHaydn_a.musicxml")
score1 = m21.converter.parse(str(score1_path))
score2_path = Path("test_scores/musicxml/longHaydn_b.musicxml")
score2 = m21.converter.parse(str(score2_path))
# build ScoreTrees
score_lin1 = nlin.Score(score1)
score_lin2 = nlin.Score(score2)

In [3]:
%%time
# compute the non common_subsequences for part 0
part = 0
seq1 = score_lin1.part_list[part].bar_list[:50]
seq2 = score_lin2.part_list[part].bar_list[:50]
print("About to compute LCS for sequences of length", len(seq1), len(seq2))
non_common_subsequences = scl.non_common_subsequences(
    seq1, seq2
)
print(non_common_subsequences)

About to compute LCS for sequences of length 50 50
[{'original': [[[[('G5', 'None', False)],4,0,[],[],[],[], [('D5', 'None', False)],4,0,[],[],[],[], [('R', 'None', False)],4,0,[],[],[],[], [('D5', 'None', False)],4,0,['start', 'start'],[],[],[], [('C5', 'None', False)],4,0,['continue', 'continue'],[],[],[], [('B4', 'None', False)],4,0,['continue', 'continue'],[],[],[], [('A4', 'None', False)],4,0,['stop', 'stop'],[],[],[]]]], 'compare_to': [[[[('G5', 'None', False)],4,0,[],[],[],[], [('D5', 'None', False)],4,0,[],[],[],[], [('R', 'None', False)],4,0,[],[],[],[], [('D5', 'sharp', False)],4,0,['start', 'start'],[],[],[], [('C5', 'None', False)],4,0,['continue', 'continue'],[],[],[], [('B4', 'None', False)],4,0,['continue', 'continue'],[],[],[], [('A4', 'None', False)],4,0,['stop', 'stop'],[],[],[]]]]}, {'original': [[[[('C5', 'sharp', False), ('G5', 'None', False)],4,0,[],[],['staccatissimo'],[], [('D5', 'None', False), ('F5', 'None', False)],4,0,[],[],['staccatissimo'],[], [('R', 'None

In [16]:
non_common_subsequences

[]

4496846402853166584